In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/catboost_model/pytorch/default/1/catboost_model.pkl
/kaggle/input/lightgbm_model/pytorch/default/1/lightgbm_model.pkl
/kaggle/input/iisc-umc-301-kaggle-competition-1/sample_submission.csv
/kaggle/input/iisc-umc-301-kaggle-competition-1/train.csv
/kaggle/input/iisc-umc-301-kaggle-competition-1/test.csv
/kaggle/input/xgboost_model/pytorch/default/1/xgboost_model.pkl


In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold
# import cudf
import optuna
import torch

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device == "cuda")

True


In [4]:
train_df = pd.read_csv('/kaggle/input/iisc-umc-301-kaggle-competition-1/train.csv')
test_df = pd.read_csv('/kaggle/input/iisc-umc-301-kaggle-competition-1/test.csv')

In [5]:
X = train_df.drop(['id', 'song_popularity'], axis=1)
y = train_df['song_popularity']
X_test = test_df.drop('id', axis=1)

print(X.shape, y.shape, X_test.shape)

test_ids = test_df['id']

(30000, 13) (30000,) (10000, 13)


In [6]:
X_train = X
y_train = y

In [14]:
X_train.isna().sum()

song_duration_ms    3067
acousticness        3024
danceability        2967
energy              2935
instrumentalness    2966
key                 3074
liveness            3022
loudness            2977
audio_mode             0
speechiness            0
tempo                  0
time_signature         0
audio_valence          0
dtype: int64

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Simple Imputation Function with Median for Numerical and Mode for Categorical

def simple_impute(X_train, X_test):
    combined = pd.concat([X_train, X_test], axis=0, ignore_index=True)

    # Separate categorical and numerical columns
    cat_cols = combined.select_dtypes(include=['int']).columns
    num_cols = combined.select_dtypes(include=['float']).columns

    # Imputers
    num_imputer = SimpleImputer(strategy="median")
    cat_imputer = SimpleImputer(strategy="most_frequent")

    # Fit only on train, transform both train and test
    X_train_num = pd.DataFrame(num_imputer.fit_transform(X_train[num_cols]),
                               columns=num_cols, index=X_train.index)
    X_test_num = pd.DataFrame(num_imputer.transform(X_test[num_cols]),
                              columns=num_cols, index=X_test.index)

    X_train_cat = pd.DataFrame(cat_imputer.fit_transform(X_train[cat_cols]),
                               columns=cat_cols, index=X_train.index)
    X_test_cat = pd.DataFrame(cat_imputer.transform(X_test[cat_cols]),
                              columns=cat_cols, index=X_test.index)

    # Combine back
    X_train_imputed = pd.concat([X_train_num, X_train_cat], axis=1)
    X_test_imputed = pd.concat([X_test_num, X_test_cat], axis=1)

    return X_train_imputed, X_test_imputed


# Example usage:
X_train_imputed, X_test_imputed = simple_impute(X_train, X_test)


In [ ]:
# Check for any remaining missing values
X_train_imputed.isna().sum()

song_duration_ms    0
acousticness        0
danceability        0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
speechiness         0
tempo               0
audio_valence       0
audio_mode          0
time_signature      0
dtype: int64

In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# Optuna Objective Function for XGBoost with imputed features

def objective_xgb(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": "gbtree",
        "tree_method": "hist",
        "device" : "cuda",
        "random_state": 42,

        # Search space
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),  # L2
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True)     # L1
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, val_idx in kf.split(X_train_imputed, y):
        X_tr, X_val = X_train_imputed.iloc[train_idx], X_train_imputed.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=2000,
            evals=[(dval, "validation")],
            early_stopping_rounds=100,
            verbose_eval=False
        )

        preds = model.predict(dval)
        aucs.append(roc_auc_score(y_val, preds))

    return sum(aucs) / len(aucs)


In [49]:
study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_xgb, n_trials=50)
print("Best params XGB:", study_xgb.best_params)
xgb_best_params = study_xgb.best_params

[I 2025-09-28 20:06:28,004] A new study created in memory with name: no-name-67646504-8eb9-4d24-b6cd-6d034cc0b25c
[I 2025-09-28 20:06:30,413] Trial 0 finished with value: 0.5673499803742186 and parameters: {'eta': 0.0373204983378169, 'max_depth': 7, 'min_child_weight': 5, 'subsample': 0.9620024585544891, 'colsample_bytree': 0.5543909360891477, 'gamma': 1.8962002233891317, 'lambda': 3.1840480670363274, 'alpha': 0.3603190512432076}. Best is trial 0 with value: 0.5673499803742186.
[I 2025-09-28 20:06:33,430] Trial 1 finished with value: 0.5699418289160005 and parameters: {'eta': 0.023894767496913393, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 0.9585784250256892, 'colsample_bytree': 0.9889069784674551, 'gamma': 0.4286526219616793, 'lambda': 0.010554456111885132, 'alpha': 0.20810727151788613}. Best is trial 1 with value: 0.5699418289160005.
[I 2025-09-28 20:06:37,453] Trial 2 finished with value: 0.5726978557689059 and parameters: {'eta': 0.016341948817392784, 'max_depth': 10, 'min

Best params XGB: {'eta': 0.02031999139412347, 'max_depth': 3, 'min_child_weight': 9, 'subsample': 0.7052949342750006, 'colsample_bytree': 0.9625032494275739, 'gamma': 3.991767496413158, 'lambda': 5.48477748251828, 'alpha': 6.903472199806726}


In [ ]:
# Optuna Objective Function for XGBoost with imputed features with a different random seed

def objective_xgb_1(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": "gbtree",
        "tree_method": "hist",
        "device" : "cuda",
        "random_state": 69,

        # Search space
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),  # L2
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True)     # L1
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, val_idx in kf.split(X_train_imputed, y):
        X_tr, X_val = X_train_imputed.iloc[train_idx], X_train_imputed.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=2000,
            evals=[(dval, "validation")],
            early_stopping_rounds=100,
            verbose_eval=False
        )

        preds = model.predict(dval)
        aucs.append(roc_auc_score(y_val, preds))

    return sum(aucs) / len(aucs)

In [40]:
study_xgb_1 = optuna.create_study(direction="maximize")
study_xgb_1.optimize(objective_xgb_1, n_trials=50)
print("Best params XGB:", study_xgb_1.best_params)
xgb_best_params_1 = study_xgb_1.best_params

[I 2025-09-28 19:52:29,516] A new study created in memory with name: no-name-2bdca75b-bf39-4479-aede-8ebacd54c4bf
[I 2025-09-28 19:52:32,222] Trial 0 finished with value: 0.5689055390037937 and parameters: {'eta': 0.02668022122056286, 'max_depth': 6, 'min_child_weight': 10, 'subsample': 0.9070573396980814, 'colsample_bytree': 0.8067121995584712, 'gamma': 0.8335949991049801, 'lambda': 0.07816171358950887, 'alpha': 0.005854106759268878}. Best is trial 0 with value: 0.5689055390037937.
[I 2025-09-28 19:52:34,064] Trial 1 finished with value: 0.5630472177102188 and parameters: {'eta': 0.11227448390439113, 'max_depth': 9, 'min_child_weight': 2, 'subsample': 0.8988445354112099, 'colsample_bytree': 0.9755700648797089, 'gamma': 1.8699531684237307, 'lambda': 9.969808731923157, 'alpha': 5.238626637572171}. Best is trial 0 with value: 0.5689055390037937.
[I 2025-09-28 19:52:35,293] Trial 2 finished with value: 0.5450506443328631 and parameters: {'eta': 0.2938320014485406, 'max_depth': 6, 'min_chi

Best params XGB: {'eta': 0.01890993341739859, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.5987836590985829, 'colsample_bytree': 0.756582037729979, 'gamma': 2.575786863576802, 'lambda': 0.03460784096516436, 'alpha': 9.71815762038713}


In [ ]:
# Training model with best hyperparameters from the second study
xgb_model_1 = xgb.train(
    xgb_best_params_1,
    dtrain,
    num_boost_round=2000,
    evals=[(dval, "validation")],
    early_stopping_rounds=100,
    verbose_eval=False
)

y_pred_xgb_1 = xgb_model_1.predict(dval)

auc_score = roc_auc_score(y_val, y_pred_xgb_1)

print(f'AUC Score: {auc_score}')

AUC Score: 0.5649209873960768


In [ ]:
# Optuna Objective Function for XGBoost with imputed features with a different random seed

def objective_xgb_2(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": "gbtree",
        "tree_method": "hist",
        "device" : "cuda",
        "random_state": 2005,

        # Search space
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),  # L2
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True)     # L1
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, val_idx in kf.split(X_train_imputed, y):
        X_tr, X_val = X_train_imputed.iloc[train_idx], X_train_imputed.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=2000,
            evals=[(dval, "validation")],
            early_stopping_rounds=100,
            verbose_eval=False
        )

        preds = model.predict(dval)
        aucs.append(roc_auc_score(y_val, preds))

    return sum(aucs) / len(aucs)

In [44]:
study_xgb_2 = optuna.create_study(direction="maximize")
study_xgb_2.optimize(objective_xgb_2, n_trials=50)
print("Best params XGB:", study_xgb_2.best_params)
xgb_best_params_2 = study_xgb_2.best_params

[I 2025-09-28 19:58:59,991] A new study created in memory with name: no-name-4cea46ec-aaec-4c3c-8722-56d672514bd4
[I 2025-09-28 19:59:05,874] Trial 0 finished with value: 0.57403063606882 and parameters: {'eta': 0.012058601017232327, 'max_depth': 8, 'min_child_weight': 9, 'subsample': 0.6790067516968068, 'colsample_bytree': 0.5874377682345255, 'gamma': 3.235502606904248, 'lambda': 0.008772571983556616, 'alpha': 5.641674814811873}. Best is trial 0 with value: 0.57403063606882.
[I 2025-09-28 19:59:10,102] Trial 1 finished with value: 0.5737124890474824 and parameters: {'eta': 0.016278323714355704, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.6966268918314699, 'colsample_bytree': 0.8497904262873321, 'gamma': 0.08813464027975115, 'lambda': 5.230377898424492, 'alpha': 3.4761003117973805}. Best is trial 0 with value: 0.57403063606882.
[I 2025-09-28 19:59:15,660] Trial 2 finished with value: 0.5725083220761474 and parameters: {'eta': 0.012189617712385914, 'max_depth': 4, 'min_child_w

Best params XGB: {'eta': 0.011736992362786906, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.5994716660770182, 'colsample_bytree': 0.5558460923525824, 'gamma': 4.559652970867748, 'lambda': 0.36940601930705785, 'alpha': 4.97102946842612}


In [ ]:
# Training model with best hyperparameters from the second study
xgb_model_2 = xgb.train(
    xgb_best_params_2,
    dtrain,
    num_boost_round=2000,
    evals=[(dval, "validation")],
    early_stopping_rounds=100,
    verbose_eval=False
)

y_pred_xgb_2 = xgb_model_2.predict(dval)

auc_score = roc_auc_score(y_val, y_pred_xgb_2)

print(f'AUC Score: {auc_score}')

AUC Score: 0.5607173407933692


In [ ]:
from catboost import CatBoostClassifier

# Optuna Objective Function for CatBoost with imputed features

def objective_cat(trial):
    params = {
        "iterations": 2000,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "random_seed": 42,
        "task_type": "GPU",    
        "verbose": False,

        # Search space
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 5.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0)
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, val_idx in kf.split(X_train_imputed, y):
        X_tr, X_val = X_train_imputed.iloc[train_idx], X_train_imputed.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]


        model = CatBoostClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=100)

        preds = model.predict_proba(X_val)[:, 1]
        aucs.append(roc_auc_score(y_val, preds))

    return sum(aucs) / len(aucs)


In [23]:
study_cat = optuna.create_study(direction="maximize")
study_cat.optimize(objective_cat, n_trials=50)
print("Best params CatBoost:", study_cat.best_params)
cat_best_params = study_cat.best_params

[I 2025-09-28 18:35:46,833] A new study created in memory with name: no-name-c8b41d8a-0080-488c-8a51-052727d577e4
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2025-09-28 18:38:30,018] Trial 0 finished with value: 0.5707440281683244 and parameters: {'depth': 9, 'learning_rate': 0.0699460582565093, 'l2_leaf_reg': 9.849043527549261, 'bagging_temperature': 4.118740474882571, 'border_count': 151, 'random_strength': 8.143033496768323}. Best is trial 0 with value: 0.5707440281683244.
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


Best params CatBoost: {'depth': 5, 'learning_rate': 0.061593611559119486, 'l2_leaf_reg': 4.670300013111426, 'bagging_temperature': 1.53721008380664, 'border_count': 236, 'random_strength': 1.6088619511834412}


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

# Optuna Objective Function for LightGBM with imputed features

def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
        "n_estimators": 2000,
        "device": "gpu"
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, val_idx in cv.split(X_train_imputed, y):
        X_tr, X_val = X_train_imputed.iloc[train_idx], X_train_imputed.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="auc",
        )
        preds = model.predict_proba(X_val)[:, 1]
        aucs.append(roc_auc_score(y_val, preds))

    return np.mean(aucs)

# Run Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50) 

print("Best trial:", study.best_trial.params)
print("Best AUC:", study.best_value)



[I 2025-09-28 19:41:13,635] A new study created in memory with name: no-name-2c6657f5-f7ae-496b-8594-c8fecd14474c
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
[I 2025-09-28 19:41:44,907] Trial 0 finished with value: 0.5615320699727822 and parameters: {'learning_rate': 0.013720218063859858, 'num_leaves': 171, 'max_depth': 10, 'min_child_samples': 28, 

KeyboardInterrupt: 

In [ ]:
# Train-Validation Split for tuning weights for ensemble
X_train, X_val, y_train, y_val = train_test_split(X_train_imputed, y, test_size=0.2, random_state=2025, stratify=y)

In [ ]:
# Train  LightGBM model on train set with best hyperparameters
lgb_best_params = study.best_params
lgb_best_params["objective"] = "binary"
lgb_best_params["metric"] = "auc"

train_data = lgb.Dataset(X_train, label=y_train)
data = lgb.Dataset(X_train_imputed, label = y)


lgb_model = lgb.train(
    lgb_best_params,
    train_data,
    num_boost_round=study.best_trial.number
)
val_preds = lgb_model.predict(X_val)
val_auc = roc_auc_score(y_val, val_preds)

print("Validation AUC:", val_auc)

In [ ]:
# Train XGBoost model on train set with best hyperparameters from second study
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
ddata = xgb.DMatrix(X_train_imputed, label = y)

In [ ]:
# Train XGBoost model on train set with best hyperparameters 
xgb_model = xgb.train(
    xgb_best_params,
    dtrain,
    num_boost_round=2000,
    evals=[(dval, "validation")],
    early_stopping_rounds=100,
    verbose_eval=False
)

In [ ]:
# Train CatBoost model on train set with best hyperparameters
cat_model = CatBoostClassifier(**cat_best_params, iterations=2000, task_type="GPU", verbose=0)
cat_model.fit(X_train, y_train)

In [ ]:
# Evaluate CatBoost on validation set
y_pred_cat = cat_model.predict_proba(X_val)[:, 1]

auc_score = roc_auc_score(y_val, y_pred_cat)

print(f'AUC Score: {auc_score}')

AUC Score: 0.5640592683286068


In [ ]:
# Evaluate XGBoost on validation set
y_pred_xgb = xgb_model.predict(dval)

auc_score = roc_auc_score(y_val, y_pred_xgb)

print(f'AUC Score: {auc_score}')

AUC Score: 0.5608554533281582


In [ ]:
# Simple weighted average ensemble
weights = [0.6, 0.4]  # [XGB, Cat]
ensemble_proba = weights[0] * y_pred_xgb + weights[1] * y_pred_cat

print("Ensemble AUC:", roc_auc_score(y_val, ensemble_proba))

Ensemble AUC: 0.5685986909114635


In [ ]:
import optuna
from sklearn.metrics import roc_auc_score
import numpy as np

# Optuna for optimizing ensemble weights
# Different ensembles of CatBoost and XGBoost models

# validation probabilities (already computed earlier)
# cat_val_proba = cat_model.predict_proba(X_val)[:, 1]
xgb_val_proba = xgb_model.predict(dval)
xgb_val_proba_1 = xgb_model_1.predict(dval)
xgb_val_proba_2 = xgb_model_2.predict(dval)

def objective(trial):
    # suggest weights (first 3), last one is 1 - sum
    # w1 = trial.suggest_float("cat_weight", 0.0, 1.0)
    w2 = trial.suggest_float("xgb_weight", 0.0, 1.0 - w1)
    w3 = trial.suggest_float("xgb1_weight", 0.0, 1.0 - w1 - w2)
    w4 = 1.0 - (w2 + w3)  # remaining weight for xgb2
    
    # weighted ensemble
    ensemble_proba = (
        # w1 * cat_val_proba +
         w2 * xgb_val_proba
        + w3 * xgb_val_proba_1
        + w4 * xgb_val_proba_2
    )
    
    # AUC with probabilities
    auc = roc_auc_score(y_val, ensemble_proba)
    return auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

best_params = study.best_params
best_auc = study.best_value

# final weights
# w1 = best_params["cat_weight"]
w2 = best_params["xgb_weight"]
w3 = best_params["xgb1_weight"]
w4 = 1.0 - (w2 + w3)

print(f"Best Weights -> XGBoost: {w2:.3f}, XGBoost_1: {w3:.3f}, XGBoost_2: {w4:.3f}")
print(f"Best Validation AUC: {best_auc:.4f}")



[I 2025-09-28 20:11:46,805] A new study created in memory with name: no-name-b75c178a-f944-417d-8bc4-5272f1602221
[I 2025-09-28 20:11:46,812] Trial 0 finished with value: 0.5607172808226637 and parameters: {'xgb_weight': 6.118242079815334e-06, 'xgb1_weight': 1.0513834476605444e-06}. Best is trial 0 with value: 0.5607172808226637.
[I 2025-09-28 20:11:46,817] Trial 1 finished with value: 0.5607173407933692 and parameters: {'xgb_weight': 5.657724361842386e-07, 'xgb1_weight': 7.844788772599764e-07}. Best is trial 1 with value: 0.5607173407933692.
[I 2025-09-28 20:11:46,822] Trial 2 finished with value: 0.5607173407933692 and parameters: {'xgb_weight': 5.518023567336722e-06, 'xgb1_weight': 5.402539089299904e-07}. Best is trial 1 with value: 0.5607173407933692.
[I 2025-09-28 20:11:46,827] Trial 3 finished with value: 0.5607168610277252 and parameters: {'xgb_weight': 1.6368922549994468e-06, 'xgb1_weight': 3.2375057307679795e-06}. Best is trial 1 with value: 0.5607173407933692.
[I 2025-09-28 2

Best Weights -> XGBoost: 0.000, XGBoost_1: 0.000, XGBoost_2: 1.000
Best Validation AUC: 0.5607


In [ ]:
# Training model with best hyperparameters and full data
xgb_model = xgb.train(
    xgb_best_params,
    ddata,
    num_boost_round=2000,
    evals=[(dval, "validation")],
    early_stopping_rounds=100,
    verbose_eval=False
)
cat_model.fit(X_train_imputed, y)
# lgb_model = lgb.train(
#     lgb_best_params,
#     data,
#     num_boost_round=study.best_trial.number
# )

In [ ]:
# Making predictions on the test set

dtest = xgb.DMatrix(X_test_imputed)
# test_data = lgb.Dataset(X_test_imputed)
y_test_prob_xgb = xgb_model.predict(dtest)
y_test_prob_cat = cat_model.predict_proba(X_test_imputed)[:, 1]
# y_test_prob_lgb= lgb_model.predict(X_test_imputed)
weights = [w1, w2]
en_prob = weights[0]*y_test_prob_cat + weights[1]*y_test_prob_xgb

In [38]:
results_df_1 = pd.DataFrame({
    'id' : test_ids,
    'song_popularity': en_prob
})
results_df_1.to_csv('/kaggle/working/predictions_prob_en_4.csv', index=False)

In [ ]:
import joblib

joblib.dump(cat_model, "catboost_model.pkl")
joblib.dump(lgb_model, "lightgbm_model.pkl")
joblib.dump(xgb_model, "xgboost_model.pkl")

In [9]:
X = X_train_imputed

In [ ]:
import numpy as np
import optuna
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


# Optuna hyperparameter tuning for XGBoost
def tune_xgb(X, y, seed, n_trials=50, n_splits=5):

    def objective(trial):
        params = {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "tree_method": "hist",
            "device" : "cuda",
            "seed": seed,
            "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        }

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
        aucs = []
        for train_idx, val_idx in kf.split(X):
            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx])
            dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx])
            bst = xgb.train(params, dtrain, num_boost_round=500,
                            evals=[(dval, "valid")],
                            early_stopping_rounds=50,
                            verbose_eval=False)
            preds = bst.predict(dval)
            aucs.append(roc_auc_score(y.iloc[val_idx], preds))

        return np.mean(aucs)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_params, study.best_value


# Tune and get OOF predictions for 10 seeds
def get_oof_predictions(X, y, seeds = [42, 123, 2021, 666, 69, 314, 2718, 420, 80085, 88888]):
    oof_preds = []
    models = []

    for seed in seeds:
        print(f"\n🔹 Tuning model with seed {seed}")
        best_params, best_auc = tune_xgb(X, y, seed)
        print(f"  Best CV AUC for seed {seed}: {best_auc:.4f}")

        # Generate OOF predictions
        kf = KFold(n_splits=5, shuffle=True, random_state=seed)
        preds = np.zeros(len(y))
        fold_models = []

        for train_idx, val_idx in kf.split(X):
            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx])
            dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx])

            bst = xgb.train(best_params, dtrain, num_boost_round=2000,
                            evals=[(dval, "valid")],
                            early_stopping_rounds=100,
                            verbose_eval=False)
            preds[val_idx] = bst.predict(dval)
            fold_models.append(bst)

        oof_preds.append(preds)
        models.append((best_params, fold_models))

    return np.array(oof_preds), models


# Optimize ensemble weights using Optuna
def optimize_weights(oof_preds, y, n_trials=200):
    n_models = oof_preds.shape[0]

    def objective(trial):
        # sample positive weights
        raw_weights = [trial.suggest_float(f"w{i}", 1e-3, 1.0) for i in range(n_models)]
        weights = np.array(raw_weights) / np.sum(raw_weights)  # normalize to sum=1

        ensemble_proba = np.average(oof_preds, axis=0, weights=weights)
        return roc_auc_score(y, ensemble_proba)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # normalize best weights
    best_weights = np.array([study.best_params[f"w{i}"] for i in range(n_models)])
    best_weights /= np.sum(best_weights)

    return best_weights, study.best_value



# Run the entire pipeline
def run_pipeline(X, y, seeds = [42, 123, 2021, 666, 69, 314, 2718, 420, 80085, 88888]):
    oof_preds, models = get_oof_predictions(X, y, seeds=seeds)

    best_weights, best_auc = optimize_weights(oof_preds, y)
    print("\n✅ Best ensemble weights:", best_weights)
    print(f"✅ Best OOF AUC: {best_auc:.4f}")

    return models, best_weights
    
models, best_weights = run_pipeline(X, y)

[I 2025-09-28 21:01:20,544] A new study created in memory with name: no-name-4affd0b4-e2e9-4645-96a0-82e0441f3cca



🔹 Tuning model with seed 42


[I 2025-09-28 21:01:27,763] Trial 0 finished with value: 0.558233368826769 and parameters: {'eta': 0.01330344111791324, 'max_depth': 10, 'subsample': 0.9809085042857588, 'colsample_bytree': 0.8212569737676096, 'lambda': 0.7916005609017175, 'alpha': 0.013039198937165312}. Best is trial 0 with value: 0.558233368826769.
[I 2025-09-28 21:01:30,805] Trial 1 finished with value: 0.5685373535059917 and parameters: {'eta': 0.01553628295658554, 'max_depth': 6, 'subsample': 0.9140899030976936, 'colsample_bytree': 0.6276732617313133, 'lambda': 1.6011939759374703, 'alpha': 1.8123599141944062}. Best is trial 1 with value: 0.5685373535059917.
[I 2025-09-28 21:01:34,079] Trial 2 finished with value: 0.5694218655550317 and parameters: {'eta': 0.010634606077393834, 'max_depth': 4, 'subsample': 0.7948028585148319, 'colsample_bytree': 0.933474906041774, 'lambda': 0.006073586845533872, 'alpha': 1.0854294151455135}. Best is trial 2 with value: 0.5694218655550317.
[I 2025-09-28 21:01:35,066] Trial 3 finishe

  Best CV AUC for seed 42: 0.5730


[I 2025-09-28 21:03:35,668] A new study created in memory with name: no-name-1199f748-be83-40a1-a7f5-44223ff70a74


...


🔹 Tuning model with seed 88888


[I 2025-09-28 21:21:29,136] Trial 0 finished with value: 0.5637796256550152 and parameters: {'eta': 0.015669889082837455, 'max_depth': 8, 'subsample': 0.9979946177424042, 'colsample_bytree': 0.655136528472921, 'lambda': 5.001830914543507, 'alpha': 0.12419660978261943}. Best is trial 0 with value: 0.5637796256550152.
[I 2025-09-28 21:21:31,387] Trial 1 finished with value: 0.5696299109135486 and parameters: {'eta': 0.04748130851651761, 'max_depth': 3, 'subsample': 0.8214234993708447, 'colsample_bytree': 0.8665134485238755, 'lambda': 0.08225106153202064, 'alpha': 0.0012918571963888607}. Best is trial 1 with value: 0.5696299109135486.
[I 2025-09-28 21:21:33,902] Trial 2 finished with value: 0.5685165221171593 and parameters: {'eta': 0.011266782021165224, 'max_depth': 7, 'subsample': 0.7278920007095913, 'colsample_bytree': 0.5022592231029479, 'lambda': 0.8954376548930718, 'alpha': 0.001277780353416344}. Best is trial 1 with value: 0.5696299109135486.
[I 2025-09-28 21:21:35,962] Trial 3 fin

  Best CV AUC for seed 88888: 0.5726


[I 2025-09-28 21:23:32,581] A new study created in memory with name: no-name-568f0d47-da82-43ca-8d4d-170a1b96620d
[I 2025-09-28 21:23:32,628] Trial 0 finished with value: 0.574861175298742 and parameters: {'w0': 0.38857598960543316, 'w1': 0.3399971997442847, 'w2': 0.21918686855373978, 'w3': 0.09583710200683548, 'w4': 0.3777455990294533, 'w5': 0.45390817792404736, 'w6': 0.670997493063031, 'w7': 0.7460001017278135, 'w8': 0.034544481460009, 'w9': 0.8629510896657625}. Best is trial 0 with value: 0.574861175298742.
[I 2025-09-28 21:23:32,637] Trial 1 finished with value: 0.5751566926571884 and parameters: {'w0': 0.4956751547520541, 'w1': 0.04235452356558386, 'w2': 0.10244021740930061, 'w3': 0.605114341683204, 'w4': 0.3591627069284144, 'w5': 0.8660307612882, 'w6': 0.1132050852342495, 'w7': 0.6007646462431175, 'w8': 0.6668801628373022, 'w9': 0.15556127607552875}. Best is trial 1 with value: 0.5751566926571884.
[I 2025-09-28 21:23:32,647] Trial 2 finished with value: 0.5755128140770824 and par


✅ Best ensemble weights: [0.04034482 0.0139854  0.24325555 0.00029329 0.02574475 0.12341567
 0.17755083 0.08873573 0.21436837 0.0723056 ]
✅ Best OOF AUC: 0.5759


In [ ]:
# Final training on full data and prediction on test set

def train_full_and_predict(X, y, X_test, models, weights):
    """
    Train all models on full dataset and predict weighted probabilities on test set.
    
    models: list of (best_params, fold_models) from get_oof_predictions
    weights: optimized weights from optimize_weights
    """
    test_preds = np.zeros(len(X_test))

    for i, (best_params, _) in enumerate(models):
        print(f"🔹 Training final model {i+1}/{len(models)} with weight {weights[i]:.4f}")

        # train on full dataset
        dtrain_full = xgb.DMatrix(X, label=y)
        dtest = xgb.DMatrix(X_test)
        bst = xgb.train(best_params, dtrain_full, num_boost_round=500)

        # predict test
        preds = bst.predict(dtest)
        test_preds += weights[i] * preds

    return test_preds


final_test_preds = train_full_and_predict(X, y, X_test_imputed, models, best_weights)


🔹 Training final model 1/10 with weight 0.0403
🔹 Training final model 2/10 with weight 0.0140
🔹 Training final model 3/10 with weight 0.2433
🔹 Training final model 4/10 with weight 0.0003
🔹 Training final model 5/10 with weight 0.0257
🔹 Training final model 6/10 with weight 0.1234
🔹 Training final model 7/10 with weight 0.1776
🔹 Training final model 8/10 with weight 0.0887
🔹 Training final model 9/10 with weight 0.2144
🔹 Training final model 10/10 with weight 0.0723


In [68]:
final_test_preds

array([0.35863175, 0.28441085, 0.3115977 , ..., 0.34719753, 0.37656164,
       0.2717879 ])

In [ ]:
# Save final predictions
results_df_1 = pd.DataFrame({
    'id' : test_ids,
    'song_popularity': final_test_preds
})
results_df_1.to_csv('/kaggle/working/preds_xgb_en_1.csv', index=False)

In [ ]:
# ============================
# Imports
# ============================
import numpy as np
import optuna
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


# Optuna hyperparameter tuning for XGBoost 

def tune_xgb(X, y, seed, n_trials=50, n_splits=5):

    def objective(trial):
        params = {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "tree_method": "hist",
            "device" : "cuda",
            "seed": seed,
            "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        }

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
        aucs = []
        for train_idx, val_idx in kf.split(X):
            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx])
            dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx])
            bst = xgb.train(params, dtrain, num_boost_round=500,
                            evals=[(dval, "valid")],
                            early_stopping_rounds=50,
                            verbose_eval=False)
            preds = bst.predict(dval)
            aucs.append(roc_auc_score(y.iloc[val_idx], preds))

        return np.mean(aucs)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_params, study.best_value


# Tune and get OOF predictions for 20 seeds 
def get_oof_predictions(X, y, seeds = [i for i in range(100,120)]):
    oof_preds = []
    models = []

    for seed in seeds:
        print(f"\n🔹 Tuning model with seed {seed}")
        best_params, best_auc = tune_xgb(X, y, seed)
        print(f"  Best CV AUC for seed {seed}: {best_auc:.4f}")

        # Generate OOF predictions
        kf = KFold(n_splits=5, shuffle=True, random_state=seed)
        preds = np.zeros(len(y))
        fold_models = []

        for train_idx, val_idx in kf.split(X):
            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx])
            dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx])

            bst = xgb.train(best_params, dtrain, num_boost_round=2000,
                            evals=[(dval, "valid")],
                            early_stopping_rounds=100,
                            verbose_eval=False)
            preds[val_idx] = bst.predict(dval)
            fold_models.append(bst)

        oof_preds.append(preds)
        models.append((best_params, fold_models))

    return np.array(oof_preds), models


# Optimize ensemble weights using Optuna

def optimize_weights(oof_preds, y, n_trials=200):
    n_models = oof_preds.shape[0]

    def objective(trial):
        # sample positive weights
        raw_weights = [trial.suggest_float(f"w{i}", 1e-3, 1.0) for i in range(n_models)]
        weights = np.array(raw_weights) / np.sum(raw_weights)  # normalize to sum=1

        ensemble_proba = np.average(oof_preds, axis=0, weights=weights)
        return roc_auc_score(y, ensemble_proba)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # normalize best weights
    best_weights = np.array([study.best_params[f"w{i}"] for i in range(n_models)])
    best_weights /= np.sum(best_weights)

    return best_weights, study.best_value


# Run the entire pipeline

def run_pipeline(X, y, seeds = [i for i in range(100, 120)]):
    oof_preds, models = get_oof_predictions(X, y, seeds=seeds)

    best_weights, best_auc = optimize_weights(oof_preds, y)
    print("\n✅ Best ensemble weights:", best_weights)
    print(f"✅ Best OOF AUC: {best_auc:.4f}")

    return models, best_weights
    
models_1, best_weights_1 = run_pipeline(X, y)

[I 2025-09-29 09:38:36,633] A new study created in memory with name: no-name-0bd5184a-7bf0-49af-a7f4-c9e2b6f06261



🔹 Tuning model with seed 100


[I 2025-09-29 09:38:40,065] Trial 0 finished with value: 0.5648686100510958 and parameters: {'eta': 0.030473081975722623, 'max_depth': 8, 'subsample': 0.5255587668241835, 'colsample_bytree': 0.7248685674756914, 'lambda': 0.03647654230119529, 'alpha': 0.4021656590736721}. Best is trial 0 with value: 0.5648686100510958.
[I 2025-09-29 09:38:41,639] Trial 1 finished with value: 0.5505895557269569 and parameters: {'eta': 0.14824473269142072, 'max_depth': 8, 'subsample': 0.8058231159342006, 'colsample_bytree': 0.7661944972132335, 'lambda': 0.1742223331631397, 'alpha': 0.0030248444300993815}. Best is trial 0 with value: 0.5648686100510958.
[I 2025-09-29 09:38:44,107] Trial 2 finished with value: 0.5729659193164861 and parameters: {'eta': 0.04190027796418442, 'max_depth': 4, 'subsample': 0.7108060763604245, 'colsample_bytree': 0.8338413320911064, 'lambda': 3.079643409410464, 'alpha': 8.0936273091158}. Best is trial 2 with value: 0.5729659193164861.
[I 2025-09-29 09:38:47,241] Trial 3 finished 

  Best CV AUC for seed 100: 0.5758


[I 2025-09-29 09:40:57,152] A new study created in memory with name: no-name-12cfc8fa-ad32-40e3-a1e6-389ec0cbf006


...


🔹 Tuning model with seed 119


[I 2025-09-29 10:19:13,836] Trial 0 finished with value: 0.5749865500273248 and parameters: {'eta': 0.03240796872856829, 'max_depth': 4, 'subsample': 0.6436805288046679, 'colsample_bytree': 0.8863528110341167, 'lambda': 1.0378785233552354, 'alpha': 1.2362726861819553}. Best is trial 0 with value: 0.5749865500273248.
[I 2025-09-29 10:19:16,874] Trial 1 finished with value: 0.5672202015750578 and parameters: {'eta': 0.024810639143807114, 'max_depth': 8, 'subsample': 0.6260106527349889, 'colsample_bytree': 0.7445616449926069, 'lambda': 0.46059676423502366, 'alpha': 0.0012180691482829077}. Best is trial 0 with value: 0.5749865500273248.
[I 2025-09-29 10:19:19,781] Trial 2 finished with value: 0.5734175758905542 and parameters: {'eta': 0.013851296583999653, 'max_depth': 4, 'subsample': 0.6682640746074633, 'colsample_bytree': 0.6077592159035781, 'lambda': 1.7489393828360096, 'alpha': 2.8863796526345347}. Best is trial 0 with value: 0.5749865500273248.
[I 2025-09-29 10:19:21,272] Trial 3 fini

  Best CV AUC for seed 119: 0.5750


[I 2025-09-29 10:21:21,566] A new study created in memory with name: no-name-a3ac522d-2dea-4046-bb9d-7e198050f2ef
[I 2025-09-29 10:21:21,577] Trial 0 finished with value: 0.5762162826883906 and parameters: {'w0': 0.605162249355939, 'w1': 0.390433828077895, 'w2': 0.525060198538535, 'w3': 0.005472212346910943, 'w4': 0.9769889489217702, 'w5': 0.9366698236699407, 'w6': 0.2704755581748588, 'w7': 0.041174375586583344, 'w8': 0.19151719025385838, 'w9': 0.7238808488936754, 'w10': 0.6327982223555624, 'w11': 0.02845620780840957, 'w12': 0.4762970199923309, 'w13': 0.40340610725970655, 'w14': 0.4319611098280205, 'w15': 0.7834736000878606, 'w16': 0.5583189745907805, 'w17': 0.8861788325490914, 'w18': 0.8018461928232294, 'w19': 0.2917043253132774}. Best is trial 0 with value: 0.5762162826883906.
[I 2025-09-29 10:21:21,588] Trial 1 finished with value: 0.5757693535206023 and parameters: {'w0': 0.019049430147209636, 'w1': 0.031204201687808908, 'w2': 0.040328901304164444, 'w3': 0.5553422187714432, 'w4': 0


✅ Best ensemble weights: [0.09303173 0.00843225 0.06501735 0.02479404 0.10723259 0.00882944
 0.09041639 0.00486544 0.10251526 0.07371624 0.05433296 0.01982741
 0.01420956 0.07067472 0.00023103 0.10063324 0.007335   0.06766953
 0.02802049 0.05821532]
✅ Best OOF AUC: 0.5767


In [31]:
def train_full_and_predict(X, y, X_test, models, weights):
    """
    Train all models on full dataset and predict weighted probabilities on test set.
    
    models: list of (best_params, fold_models) from get_oof_predictions
    weights: optimized weights from optimize_weights
    """
    test_preds = np.zeros(len(X_test))

    for i, (best_params, _) in enumerate(models):
        print(f"🔹 Training final model {i+1}/{len(models)} with weight {weights[i]:.4f}")

        # train on full dataset
        dtrain_full = xgb.DMatrix(X, label=y)
        dtest = xgb.DMatrix(X_test)
        bst = xgb.train(best_params, dtrain_full, num_boost_round=500)

        # predict test
        preds = bst.predict(dtest)
        test_preds += weights[i] * preds

    return test_preds


final_test_preds_1 = train_full_and_predict(X, y, X_test_imputed, models_1, best_weights_1)

🔹 Training final model 1/20 with weight 0.0930
🔹 Training final model 2/20 with weight 0.0084
🔹 Training final model 3/20 with weight 0.0650
🔹 Training final model 4/20 with weight 0.0248
🔹 Training final model 5/20 with weight 0.1072
🔹 Training final model 6/20 with weight 0.0088
🔹 Training final model 7/20 with weight 0.0904
🔹 Training final model 8/20 with weight 0.0049
🔹 Training final model 9/20 with weight 0.1025
🔹 Training final model 10/20 with weight 0.0737
🔹 Training final model 11/20 with weight 0.0543
🔹 Training final model 12/20 with weight 0.0198
🔹 Training final model 13/20 with weight 0.0142
🔹 Training final model 14/20 with weight 0.0707
🔹 Training final model 15/20 with weight 0.0002
🔹 Training final model 16/20 with weight 0.1006
🔹 Training final model 17/20 with weight 0.0073
🔹 Training final model 18/20 with weight 0.0677
🔹 Training final model 19/20 with weight 0.0280
🔹 Training final model 20/20 with weight 0.0582


In [ ]:
# Save final predictions
# These predictions gave highest score on leaderboard
results_df_1 = pd.DataFrame({
    'id' : test_ids,
    'song_popularity': final_test_preds_1
})
results_df_1.to_csv('/kaggle/working/preds_xgb_en_3.csv', index=False)

In [ ]:
import numpy as np
import optuna
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import random


# Optuna hyperparameter tuning for XGBoost
def tune_xgb(X, y, seed, n_trials=50, n_splits=5):

    def objective(trial):
        params = {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "tree_method": "hist",
            "device" : "cuda",
            "seed": seed,
            "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
        }

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
        aucs = []
        for train_idx, val_idx in kf.split(X):
            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx])
            dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx])
            bst = xgb.train(params, dtrain, num_boost_round=500,
                            evals=[(dval, "valid")],
                            early_stopping_rounds=50,
                            verbose_eval=False)
            preds = bst.predict(dval)
            aucs.append(roc_auc_score(y.iloc[val_idx], preds))

        return np.mean(aucs)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_params, study.best_value


# Tune and get OOF predictions for 20 random seeds
def get_oof_predictions(X, y, seeds = [random.randint(1, 2**32 -1) for _ in range(20)]):
    oof_preds = []
    models = []

    for seed in seeds:
        print(f"\n🔹 Tuning model with seed {seed}")
        best_params, best_auc = tune_xgb(X, y, seed)
        print(f"  Best CV AUC for seed {seed}: {best_auc:.4f}")

        # Generate OOF predictions
        kf = KFold(n_splits=5, shuffle=True, random_state=seed)
        preds = np.zeros(len(y))
        fold_models = []

        for train_idx, val_idx in kf.split(X):
            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx])
            dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx])

            bst = xgb.train(best_params, dtrain, num_boost_round=2000,
                            evals=[(dval, "valid")],
                            early_stopping_rounds=100,
                            verbose_eval=False)
            preds[val_idx] = bst.predict(dval)
            fold_models.append(bst)

        oof_preds.append(preds)
        models.append((best_params, fold_models))

    return np.array(oof_preds), models


# Optimize ensemble weights using Optuna
def optimize_weights(oof_preds, y, n_trials=200):
    n_models = oof_preds.shape[0]

    def objective(trial):
        # sample positive weights
        raw_weights = [trial.suggest_float(f"w{i}", 1e-3, 1.0) for i in range(n_models)]
        weights = np.array(raw_weights) / np.sum(raw_weights)  # normalize to sum=1

        ensemble_proba = np.average(oof_preds, axis=0, weights=weights)
        return roc_auc_score(y, ensemble_proba)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    # normalize best weights
    best_weights = np.array([study.best_params[f"w{i}"] for i in range(n_models)])
    best_weights /= np.sum(best_weights)

    return best_weights, study.best_value



# Run the entire pipeline
def run_pipeline(X, y, seeds = [random.randint(1, 2**32 -1) for _ in range(20)]):
    oof_preds, models = get_oof_predictions(X, y, seeds=seeds)

    best_weights, best_auc = optimize_weights(oof_preds, y)
    print("\n✅ Best ensemble weights:", best_weights)
    print(f"✅ Best OOF AUC: {best_auc:.4f}")

    return models, best_weights
    
models_2, best_weights_2 = run_pipeline(X, y)

[I 2025-09-29 10:59:03,622] A new study created in memory with name: no-name-d6033d50-818b-4a05-8b33-72cbd5240fa2



🔹 Tuning model with seed 1475011102


[I 2025-09-29 10:59:06,608] Trial 0 finished with value: 0.5523220497355352 and parameters: {'eta': 0.07596937450702265, 'max_depth': 9, 'subsample': 0.9569901080937973, 'colsample_bytree': 0.6176491932955848, 'lambda': 0.0023394138739821183, 'alpha': 0.03494359392698946}. Best is trial 0 with value: 0.5523220497355352.
[I 2025-09-29 10:59:07,577] Trial 1 finished with value: 0.5649607796910071 and parameters: {'eta': 0.16562640292459255, 'max_depth': 4, 'subsample': 0.8907826672948136, 'colsample_bytree': 0.8167587757537538, 'lambda': 0.008951744233737992, 'alpha': 0.669766161012845}. Best is trial 1 with value: 0.5649607796910071.
[I 2025-09-29 10:59:08,501] Trial 2 finished with value: 0.5602829876462829 and parameters: {'eta': 0.16619550410828204, 'max_depth': 5, 'subsample': 0.6805879662273555, 'colsample_bytree': 0.5330818190537179, 'lambda': 0.004513945645034937, 'alpha': 1.526197934105216}. Best is trial 1 with value: 0.5649607796910071.
[I 2025-09-29 10:59:11,253] Trial 3 fini

  Best CV AUC for seed 1475011102: 0.5731


[I 2025-09-29 11:01:19,885] A new study created in memory with name: no-name-41daa03f-14a2-41b8-b2c5-38e3d23f783b


...

[I 2025-09-29 11:05:40,751] Trial 0 finished with value: 0.5591541160729122 and parameters: {'eta': 0.020696408461950516, 'max_depth': 10, 'subsample': 0.7345461364540087, 'colsample_bytree': 0.6749845800025824, 'lambda': 0.00367209644421308, 'alpha': 0.0012825802101665488}. Best is trial 0 with value: 0.5591541160729122.
[I 2025-09-29 11:05:42,696] Trial 1 finished with value: 0.5679100725345164 and parameters: {'eta': 0.03399867705461796, 'max_depth': 6, 'subsample': 0.9687316981894496, 'colsample_bytree': 0.8388620774233874, 'lambda': 1.0917934316262339, 'alpha': 0.0036980508072510547}. Best is trial 1 with value: 0.5679100725345164.
[I 2025-09-29 11:05:44,045] Trial 2 finished with value: 0.5464452232136336 and parameters: {'eta': 0.15396570172560434, 'max_depth': 7, 'subsample': 0.6488713448405856, 'colsample_bytree': 0.7913659519296898, 'lambda': 0.698700841588562, 'alpha': 0.0010090992661571747}. Best is trial 1 with value: 0.5679100725345164.
[I 2025-09-29 11:05:47,261] Trial 3

  Best CV AUC for seed 4209121005: 0.5743


[I 2025-09-29 11:07:57,538] A new study created in memory with name: no-name-517c9861-446f-4a72-9e2a-52344dcf4540



🔹 Tuning model with seed 1010362157


[I 2025-09-29 11:07:59,365] Trial 0 finished with value: 0.570778222643838 and parameters: {'eta': 0.060024151073821516, 'max_depth': 3, 'subsample': 0.8650128995257291, 'colsample_bytree': 0.9390179423318146, 'lambda': 0.033521315911836075, 'alpha': 0.01373154344802671}. Best is trial 0 with value: 0.570778222643838.
[I 2025-09-29 11:08:00,856] Trial 1 finished with value: 0.5662827380540578 and parameters: {'eta': 0.06410198390244845, 'max_depth': 5, 'subsample': 0.6833480411191639, 'colsample_bytree': 0.5687283242496612, 'lambda': 0.2247769287526247, 'alpha': 0.5853476627394752}. Best is trial 0 with value: 0.570778222643838.
[I 2025-09-29 11:08:05,886] Trial 2 finished with value: 0.5664458366954952 and parameters: {'eta': 0.02305024502436705, 'max_depth': 9, 'subsample': 0.7443260281980593, 'colsample_bytree': 0.5864654985206262, 'lambda': 0.4071874558830309, 'alpha': 2.644880182917451}. Best is trial 0 with value: 0.570778222643838.
[I 2025-09-29 11:08:07,224] Trial 3 finished wi